In [1]:
pip install tensorflow opencv-python matplotlib

In [4]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten




In [5]:
base= 'train_dataset/'
imgsize= 180
batch=32


In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(base,seed=123,validation_split=0.2,subset='training',batch_size=batch,image_size=(imgsize,imgsize))
val_ds = tf.keras.utils.image_dataset_from_directory(base,seed=123,validation_split=0.2,subset='validation',batch_size=batch,image_size=(imgsize,imgsize))


Found 6201 files belonging to 8 classes.
Using 4961 files for training.
Found 6201 files belonging to 8 classes.
Using 1240 files for validation.


In [7]:
terrain_types= train_ds.class_names
terrain_types

['bright dune',
 'crater',
 'dark dune',
 'impact ejecta',
 'other',
 'slope streak',
 'spider',
 'swiss cheese']

In [8]:
AUTOTUNE= tf.data.AUTOTUNE

In [9]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [10]:
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [11]:
data_aug= Sequential([
    layers.RandomFlip("horizontal", input_shape=(imgsize,imgsize,3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

model= Sequential([
    data_aug,
    layers.Rescaling(1./255),
    Conv2D(16,3, padding='same',activation='relu'),
    MaxPooling2D(),
    Conv2D(32,3, padding='same',activation='relu'),
    MaxPooling2D(),
    Conv2D(64,3, padding='same',activation='relu'),
    MaxPooling2D(),
    Conv2D(128,3, padding='same',activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(128,activation='relu'),
    Dense(8)
])

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [22]:
history=model.fit(train_ds,epochs=50,validation_data=val_ds)

Epoch 1/50


156/156 ━━━━━━━━━━━━━━━━━━━━ 94s 603ms/step - accuracy: 0.9592 - loss: 0.1219 - val_accuracy: 0.8927 - val_loss: 0.4514
Epoch 2/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 81s 519ms/step - accuracy: 0.9604 - loss: 0.1237 - val_accuracy: 0.8984 - val_loss: 0.4579
Epoch 3/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 80s 513ms/step - accuracy: 0.9545 - loss: 0.1220 - val_accuracy: 0.8944 - val_loss: 0.5032
Epoch 4/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 82s 527ms/step - accuracy: 0.9571 - loss: 0.1246 - val_accuracy: 0.8919 - val_loss: 0.4853
Epoch 5/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 80s 514ms/step - accuracy: 0.9516 - loss: 0.1298 - val_accuracy: 0.8871 - val_loss: 0.4807
Epoch 6/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 82s 528ms/step - accuracy: 0.9591 - loss: 0.1142 - val_accuracy: 0.8895 - val_loss: 0.4369
Epoch 7/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 77s 496ms/step - accuracy: 0.9627 - loss: 0.1112 - val_accuracy: 0.8960 - val_loss: 0.4670
Epoch 8/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 81s 522ms/step - accuracy: 0.9584 - loss: 0.1123 - val

In [23]:
def classify_terrain(img_path):
    input_image=tf.keras.utils.load_img(img_path,target_size=(180,180))
    input_image_array = tf.keras.utils.img_to_array(input_image)
    inp_image_exp_dim = tf.expand_dims(input_image_array,0)
    prediction = model.predict(inp_image_exp_dim)
    result = tf.nn.softmax(prediction[0])
    return terrain_types[np.argmax(result)]

In [24]:
classify_terrain('test_dataset/test_dataset/8.jpg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step


'other'

In [ ]:
model.save('Terrain_Classifier.keras')

In [25]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (1, 180, 180, 3)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (1, 180, 180, 3)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (1, 180, 180, 16)      │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (1, 90, 90, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (1, 90, 90, 32)        │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (1, 45, 45, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (1, 45, 45, 64)        │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (1, 22, 22, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (1, 22, 22, 128)       │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (1, 11, 11, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (1, 11, 11, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (1, 15488)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1, 128)               │     1,982,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (1, 8)                 │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,243,194 (23.82 MB)

 Trainable params: 2,081,064 (7.94 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,162,130 (15.88 MB)

In [29]:
import csv

In [30]:
test_dir = 'test_dataset/test_dataset'
image_files = [f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Classify each image and store the results
results = []
for img_file in image_files:
    img_path = os.path.join(test_dir, img_file)
    classification = classify_terrain(img_path)
    results.append((img_file, classification))

# Save the results to a CSV file
with open('test.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'Class'])
    writer.writerows(results)

print("Classification results saved to test.csv")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━